In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: 1
    Vertex: body2, Edge: 2

In [6]:
x = MechanismState{Float64}(mechanism);

In [7]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "1": Quaternion f… => [0.34756776393956357,0.3444302352973123,0.2288838…
  Joint "2": Revolute joi… => [0.05025053387516376]

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
cache = MechanismStateCache(mechanism, x);

In [11]:
relative_transform(cache, joint2.frameAfter, body1.frame)

Transform3D from "after_2" to "body1":
rotation: 2.2654990618705444 rad about [0.22213693098748205,0.7637497705222392,0.6060870167877623], translation: Vec(0.9860216976315936,0.5285057387625262,0.889891123968227)


In [12]:
center_of_mass(cache)

Point3D in "world": Vec(1.7254468122990019,0.36473462851844834,1.1085075078880327)

In [13]:
mass(mechanism)

1.7499386209086778

In [14]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(1.3345270473400428,0.649292925725877,1.299660610882931)

In [15]:
relative_twist(cache, body2, body1)

Twist of "after_2" w.r.t "after_1" in "world":
angular: Vec(0.036490982002168125,0.026484492236902066,0.016577454023931182), linear: Vec(-0.023657227735251862,0.02530283118968367,0.01165093475841511)

In [16]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [17]:
geometric_jacobian(cache, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_2", expressed in "world":
6x7 Array{Float64,2}:
 -0.726181  -0.772029    0.58777   …  -0.0       -0.0       -0.0     
 -0.527049  -0.127785   -0.516304     -0.0       -0.0       -0.0     
 -0.329896  -0.622609   -0.622862     -0.0       -0.0       -0.0     
  0.470786  -0.514219   -0.371881     -0.772029   0.58777    0.241864
 -0.503534  -0.0810284   0.418026     -0.127785  -0.516304   0.846818
 -0.231857   0.654257   -0.697441  …  -0.622609  -0.622862  -0.47371 

In [18]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  1.26798    0.827343     -1.69986   -1.11022e-16  …  -1.11702       0.286422
  0.390835   4.02446       0.231492   0.816946        -2.58017       1.18514 
 -1.64698    0.981688      2.75596    1.11702          6.93889e-17  -0.843053
  0.0        0.816946      1.11702    1.74994          1.11022e-16  -0.386179
 -0.816946  -1.11022e-16   2.58017   -1.11022e-16      2.22045e-16  -0.381266
 -1.11702   -2.58017       0.0        1.11022e-16  …   1.74994      -0.732676
  0.286422   1.18514      -0.843053  -0.386179        -0.732676      0.171059

In [19]:
A = momentum_matrix(cache)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
  0.0197596  -3.44734   -0.581062   0.149509  -0.603988   1.94503   -1.15239 
  2.37105     2.04451   -3.41957   -0.382325  -3.02085   -1.8995     1.66827 
 -0.253254    3.09974    1.46007   -0.106922   1.93409   -2.40253    0.54575 
  0.750343    1.25476   -0.654176   1.351     -1.02856   -0.423247   0.103163
  0.52412     2.28933    1.47489    0.223615   0.903501  -1.48188    0.374246
 -1.03799    -0.713616   2.30256    1.08953    1.08997    0.828963  -0.82499 

In [20]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(-1.261142129688028,-0.1704889039248208,1.4986093616111562), linear: Vec(0.43383076459432607,1.3940546790055222,0.38489948280728675)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(cache, accel, world.frame)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(0.14680948425834056,0.059528457241137506,0.23992735521094496), linear: Vec(0.2964107266620637,0.45317459338943833,0.35004049767883566)

In [22]:
bias_acceleration(cache, body2)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(-0.013813408233942667,0.01950709675109126,-0.0007583022819562229), linear: Vec(-0.02317193617585795,-0.009141761508438796,0.016657855131553483)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(cache, v̇)

7-element Array{Float64,1}:
   9.3776 
  10.4197 
 -19.6353 
 -10.083  
  -8.37031
 -10.3064 
   8.27866

In [24]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [25]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
    Vertex: body4, Edge: joint4
  Vertex: body3, Edge: joint3
    Vertex: body5, Edge: joint5
      Vertex: body6, Edge: joint6
      Vertex: body7, Edge: joint7
        Vertex: body9, Edge: joint9
      Vertex: body8, Edge: joint8
      Vertex: body11, Edge: joint11
    Vertex: body10, Edge: joint10